In [ ]:
import os.path
import datetime
import numpy
import pygetm

# Note: "%matplotlib widget" below enables interactive plots but requires https://github.com/matplotlib/ipympl
# Alternatively you could use "%matplotlib notebook" (deprecated).
%matplotlib widget
import matplotlib.pyplot

In [ ]:
# Set up rectangular domain with outer points masked
domain = pygetm.domain.create_cartesian(500.*numpy.arange(131), 500.*numpy.arange(3), 20, lat=0, H=20)
sim = pygetm.Simulation(domain,
                        runtype=pygetm.BAROCLINIC, 
                        advection_scheme=pygetm.AdvectionScheme.HSIMT,
                        airsea=pygetm.airsea.Fluxes()
)

# set mask all around the basin

sim.temp.fill(20.)
print(domain.x)
sim.salt.fill(numpy.where(domain.T.x.all_values<32500., 20, 30))

# Idealized surface forcingA - likely not needed because of airsea initialization
#tausx = domain.U.array(fill=0.0)
#tausy = domain.V.array(fill=0.)
#sp = domain.T.array(fill=0.)
#fwf = domain.T.array(fill=0.)

#?
#idpdx = domain.U.array(fill=0., z=pygetm.CENTERS)
#idpdy = domain.V.array(fill=0., z=pygetm.CENTERS)
#viscosity = domain.T.array(fill=1e-1, z=pygetm.INTERFACES)


# is this needed?
domain.T.zio.all_values[...] = 0
domain.T.zin.all_values[...] = 0

sim.start_3d()

# Set up velocity plot
# want to make a vertical slice animation of either S or rho

#nsample_x = 2
#nsample_z = 2
#iy = 0
#fig, ax = matplotlib.pyplot.subplots()
#u = sim.pk.interp(domain.T) / domain.T.H
#w = sim.ww.interp(domain.T, z=pygetm.CENTERS)
#z_2d = domain.T.zc[::nsample_z, iy, ::nsample_x]
#x_2d = numpy.broadcast_to(domain.T.x[iy, ::nsample_x], z_2d.shape)
#Q = ax.quiver(x_2d, z_2d, u[::nsample_z, iy, ::nsample_x], w[::nsample_z, iy, ::nsample_x], scale=.01)
#title = ax.set_title('time: 0 s')

In [ ]:
ntime = 24*int(3600. // timestep)
sim.start(datetime.datetime(2015, 1, 1), timestep=15., split_factor=20, report=180)

times = timestep * numpy.arange(ntime)
plotting_interval = 5
for istep, time in enumerate(times):
    sim.advance()
    # update plot
    #if istep % plotting_interval == 0:
    #    ......

sim.finish()

# and drop this
#    sim.update_surface_pressure_gradient(domain.T.z, sp)
#    sim.update_2d_momentum(timestep, tausx, tausy, sim.dpdx, sim.dpdy)
#    sim.update_sealevel(timestep, sim.U, sim.V, fwf)
#    sim.update_depth()

#    if istep % mode_split == 0:
#        sim.Ui.all_values[...] /= mode_split
#        sim.Vi.all_values[...] /= mode_split
#        sim.start_3d()
#        sim.update_surface_pressure_gradient(domain.T.zio, sp)
#        sim.update_3d_momentum(timestep * mode_split, tausx, tausy, sim.dpdx, sim.dpdy, idpdx, idpdy, viscosity)
#        sim.Ui.all_values[...] = 0
#        sim.Vi.all_values[...] = 0

    # Update velocity plot
    #if istep % plotting_interval == 0:
     #   u = sim.pk.interp(domain.T) / domain.T.H
      #  sim.ww.interp(w)
       # Q.set_UVC(u[::nsample_z, iy, ::nsample_x], w[::nsample_z, iy, ::nsample_x])
        #title.set_text('time: %s s' % time)
       # fig.canvas.draw()